In [1]:
import polars as pl
import numpy as np
import random
from datetime import datetime, timedelta

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [2]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [3]:
num_cars = 1000
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
print(data.head())

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SLICE[offset: 0, len: 5]
  DF ["car_id", "timestamp", "latitude", "longitude"]; PROJECT */5 COLUMNS; SELECTION: None


## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [4]:

d2 = data
print(d2.collect())


shape: (6_287_430, 5)
┌─────────┬────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id  ┆ timestamp                  ┆ latitude  ┆ longitude   ┆ notification  │
│ ---     ┆ ---                        ┆ ---       ┆ ---         ┆ ---           │
│ str     ┆ str                        ┆ f64       ┆ f64         ┆ str           │
╞═════════╪════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0   ┆ 2023-01-01T00:00:00        ┆ 18.14631  ┆ -94.794521  ┆ tire_pressure │
│ car_0   ┆ 2023-01-01T00:02:21.273346 ┆ 18.14631  ┆ -94.794521  ┆ null          │
│ car_0   ┆ 2023-01-01T00:12:53.361581 ┆ 18.14631  ┆ -94.794521  ┆ engine_check  │
│ car_0   ┆ 2023-01-01T00:11:04.827178 ┆ 18.14631  ┆ -94.794521  ┆ tire_pressure │
│ car_0   ┆ 2023-01-01T00:00:46.171553 ┆ 18.14631  ┆ -94.794521  ┆ engine_check  │
│ …       ┆ …                          ┆ …         ┆ …           ┆ …             │
│ car_999 ┆ 2023-01-07T23:28:03.738720 ┆ 28.798342 ┆ -114.705792 

In [5]:
d2 = d2.with_columns(
    pl.when(pl.col("timestamp").str.contains(r"\."))
    .then(pl.col("timestamp"))
    .otherwise(pl.col("timestamp") + ".000000")
    .alias("timestamp")
)
#Al no tener todos los valores de la columna timestamps en el mismo formato
#aplicamos el código dearriba para poner todos los segundos sin fracción o decimal para cambiar el tipo

d2 = d2.with_columns(
    pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%f", strict=True).alias("timestamp")
)




/var/folders/tl/d786gk2904gf5nsvpr28ss000000gn/T/ipykernel_35948/1458397175.py:11: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%f", strict=True).alias("timestamp")


In [6]:

d2.collect()

car_id,timestamp,latitude,longitude,notification
str,datetime[μs],f64,f64,str
"""car_0""",2023-01-01 00:00:00,18.14631,-94.794521,"""tire_pressure"""
"""car_0""",2023-01-01 00:02:21.000273,18.14631,-94.794521,null
"""car_0""",2023-01-01 00:12:53.000361,18.14631,-94.794521,"""engine_check"""
"""car_0""",2023-01-01 00:11:04.000827,18.14631,-94.794521,"""tire_pressure"""
"""car_0""",2023-01-01 00:00:46.000171,18.14631,-94.794521,"""engine_check"""
…,…,…,…,…
"""car_999""",2023-01-07 23:28:03.000738,28.798342,-114.705792,"""tire_pressure"""
"""car_999""",2023-01-07 23:23:25.000837,28.798342,-114.705792,"""tire_pressure"""
"""car_999""",2023-01-07 23:44:56.000854,28.802843,-114.707076,"""tire_pressure"""


### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es encesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [7]:

df_aux = d2.group_by("car_id").agg(
          pl.col("timestamp")
          .sort_by("timestamp")
          .diff()
          .alias("notif_time")
      )
d2 = d2.join(df_aux, on="car_id", how="left")


In [8]:
print(df_aux.collect())

shape: (1_000, 2)
┌─────────┬─────────────────────────────────┐
│ car_id  ┆ notif_time                      │
│ ---     ┆ ---                             │
│ str     ┆ list[duration[μs]]              │
╞═════════╪═════════════════════════════════╡
│ car_938 ┆ [null, 4s 621µs, … 5m 52s 9994… │
│ car_969 ┆ [null, 2m 11s 831µs, … 2m 23s … │
│ car_901 ┆ [null, 9m 36s 516µs, … 4m 10s … │
│ car_702 ┆ [null, 1m 35s 878µs, … 9m 5s 1… │
│ car_634 ┆ [null, 5m 29s 314µs, … 4m 15s … │
│ …       ┆ …                               │
│ car_896 ┆ [null, 57s 974µs, … 11m 35s 25… │
│ car_81  ┆ [null, 48s 11µs, … 34s 999447µ… │
│ car_935 ┆ [null, 30m, … 20m 19s 999503µs… │
│ car_863 ┆ [null, 5m 3s 171µs, … 53s 9992… │
│ car_130 ┆ [null, 6m 48s 559µs, … 1m 19s … │
└─────────┴─────────────────────────────────┘


2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [9]:
#Funcion auxiliar para calcular la distancia con latitud y longitud
import math
def haversine(lat1, lon1, lat2, lon2):
    # Convertir las coordenadas de grados a radianes
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Diferencias
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Radio de la Tierra (en kilómetros)
    R = 6371.0
    
    # Distancia en kilómetros
    distance = R * c
    return distance

In [10]:
d2 = (
    d2
    .sort(["car_id", "timestamp"])  # Ordenar por vehículo y tiempo
    .with_columns(
        pl.col("latitude").shift(1).over("car_id").alias("prev_latitude"),  # Latitud previa
        pl.col("longitude").shift(1).over("car_id").alias("prev_longitude")  # Longitud previa
    )
    .with_columns(
        pl.struct(["latitude", "longitude", "prev_latitude", "prev_longitude"]).map_elements(
            lambda row: haversine(
                row["latitude"], row["longitude"], row["prev_latitude"], row["prev_longitude"]
            ) if row["prev_latitude"] is not None else None
        , return_dtype=pl.Float64).alias("distance")
    )
    .drop(["prev_latitude", "prev_longitude"])
)


3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`


In [22]:
x = 55
df_intervals = data.clone()
df_intervals = (
    df_intervals
    .sort(["car_id", "timestamp"])
    .group_by_dynamic(
        index_column="timestamp",
        every=f"{x}m",
        by="car_id",
        include_boundaries=True
    )
    .agg([
        pl.col("notification_time").mean().alias("mean_notification_time"),
        pl.col("notification_time").median().alias("median_notification_time"),
        pl.col("notification_time").var().alias("variance_notification_time"),
        pl.col("notification_time").max().alias("max_notification_time"),
        pl.col("notification_time").min().alias("min_notification_time"),
        pl.col("distance").mean().alias("mean_distance"),
        pl.col("distance").median().alias("median_distance"),
        pl.col("distance").var().alias("variance_distance"),
        pl.col("distance").max().alias("max_distance"),
        pl.col("distance").min().alias("min_distance"),
    ])
)


/var/folders/tl/d786gk2904gf5nsvpr28ss000000gn/T/ipykernel_35948/3383310550.py:6: DeprecationWarning: The argument `by` for `LazyFrame.group_by_dynamic` is deprecated. It has been renamed to `group_by`.
  .group_by_dynamic(
